## The following section is for Colab Users.
### Just run the following code cells

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 65.9 MB/s 
     |████████████████████████████████| 154 kB 75.1 MB/s 


In [ ]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar -P /content/spark-3.0.2-bin-hadoop2.7/jars/
!cp /content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.2-bin-hadoop2.7/graphframes-0.8.2-spark3.0-s_2.12.zip

--2022-06-28 11:43:07--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.225.142.29, 13.225.142.128, 13.225.142.70, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.225.142.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247882 (242K) [binary/octet-stream]
Saving to: ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.02s   

2022-06-28 11:43:07 (12.0 MB/s) - ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’ saved [247882/247882]



In [1]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [2]:
import findspark
findspark.init()

In [3]:
!export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"
!export PYSPARK_DRIVER_PYTHON=jupyter
!export PYSPARK_DRIVER_PYTHON_OPTS=notebook

In [4]:
from pyspark.sql import SparkSession
from graphframes import *

spark = SparkSession.builder.master("local[*]").appName("GraphFrames").getOrCreate()

In [5]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

**************************************************************************
**************************************************************************
**************************************************************************

In [6]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read departuredelays.csv in Edge DataFrame
### Read airport-codes-na.txt in Vertix DataFrame (the separator is Tab i.e sep = '\t' )

#### The US flight delays data set has five columns:
- The <b>date</b> column contains an integer like 02190925 . When converted, this maps to 02-19 09:25 am.
- The <b>delay</b> column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The <b>distance</b> column gives the distance in miles from the origin airport to the destination airport.
- The <b>origin</b> column contains the origin IATA airport code.
- The <b>destination</b> column contains the destination IATA airport code.

#### The airport-codes data set has four columns:
- The <b>IATA</b> column contains IATA airport code.
- The <b>City, State, and Country</b> columns contains information about the airport location. 

In [7]:
vertix = spark.read.csv("airport-codes-na.txt" , header= True ,
                                    inferSchema =True ,sep = "\t")

In [8]:
vertix.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [9]:
edge = spark.read.csv('departuredelays.csv' ,header=True , inferSchema =True)
edge.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [10]:
edge.show(20)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

### In the vertix DataFrame, drop any duplicated rows with the same  IATA code.

In [11]:
vertix.dropDuplicates()
vertix.show(10)

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+
only showing top 10 rows



### In the edges DataFrame:
- Rename the <b>date</b> columns to become <b>tripid</b>.
- Rename the <b>origin</b> columns to become <b>src</b>.
- Rename the <b>destination</b> columns to become <b>dst</b>.

In [12]:
edge1 = edge.withColumnRenamed("date","tripid").withColumnRenamed("origin","src").withColumnRenamed('destination' , 'dst')
edge1.show(10)

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
+-------+-----+--------+---+---+
only showing top 10 rows



### In the Vertix DataFrame:
- Rename the <b>IATA</b> columns to become <b>id</b>.

In [13]:
vertix1 = vertix.withColumnRenamed("IATA","id")
vertix1.show(5)

+----------+-----+-------+---+
|      City|State|Country| id|
+----------+-----+-------+---+
|Abbotsford|   BC| Canada|YXX|
|  Aberdeen|   SD|    USA|ABR|
|   Abilene|   TX|    USA|ABI|
|     Akron|   OH|    USA|CAK|
|   Alamosa|   CO|    USA|ALS|
+----------+-----+-------+---+
only showing top 5 rows



In [14]:
vertix1.cache()

DataFrame[City: string, State: string, Country: string, id: string]

### Create GraphFrame from Vertix and Edges DataFrames

In [15]:
airportGraph = GraphFrame(vertix1, edge1)
print(airportGraph)

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 3 more fields])


### Determine the number of airports

In [16]:
print(f"Airports count =  {airportGraph.vertices.count()}")

Airports count =  526


### Determine the number of trips 

In [17]:
print(f"Trips = {airportGraph.edges.count()}")

Trips = 1391578


### What is the longest delay?

In [18]:
longest = airportGraph.edges.groupBy().max("delay")
longest.show()

+----------+
|max(delay)|
+----------+
|      1642|
+----------+



### Find out the number of delayed flights vs. early flights (flights that departed before actual time)

In [19]:
print(f"Ontime - Early Flights : " , airportGraph.edges.filter("delay <= 0").count())
print(f"Delayed Flights : " , airportGraph.edges.filter("delay > 0").count())

Ontime - Early Flights :  799851
Delayed Flights :  591727


### What flight destinations departing SFO are most likely to have significant delays? Select the top 10
#### Hint: you should get the average delay for each destination for trips that depart from SFO only

In [20]:
from pyspark.sql.functions import *
sfo1 = airportGraph.edges.filter("src = 'SFO' and delay > 0").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)"))
sfo1.show(10)

+---+---+------------------+
|src|dst|        avg(delay)|
+---+---+------------------+
|SFO|OKC|59.073170731707314|
|SFO|JAC| 57.13333333333333|
|SFO|COS|53.976190476190474|
|SFO|OTH| 48.09090909090909|
|SFO|SAT|            47.625|
|SFO|MOD| 46.80952380952381|
|SFO|SUN|46.723404255319146|
|SFO|CIC| 46.72164948453608|
|SFO|ABQ|           44.8125|
|SFO|ASE|44.285714285714285|
+---+---+------------------+
only showing top 10 rows



##Incoming and outgoing connections

In [21]:
airportGraph.degrees.orderBy(desc("degree")).show(20)

+---+------+
| id|degree|
+---+------+
|ATL|181918|
|DFW|134532|
|ORD|126195|
|LAX|107687|
|DEN|104069|
|IAH| 86061|
|PHX| 79876|
|SFO| 78471|
|LAS| 66101|
|CLT| 56790|
|MCO| 56272|
|EWR| 55308|
|SLC| 51191|
|LGA| 50927|
|BOS| 50708|
|JFK| 47056|
|DTW| 46731|
|MSP| 46416|
|SEA| 46152|
|MIA| 43622|
+---+------+
only showing top 20 rows



### Find the Incoming connections to the airport sorted in Desc. order.

In [22]:
airportGraph.inDegrees.orderBy(col('inDegree').desc()).show(20)

+---+--------+
| id|inDegree|
+---+--------+
|ATL|   90434|
|DFW|   66050|
|ORD|   61967|
|LAX|   53601|
|DEN|   50921|
|IAH|   42700|
|PHX|   39721|
|SFO|   38988|
|LAS|   32994|
|CLT|   28388|
|MCO|   27959|
|EWR|   27652|
|LGA|   25469|
|BOS|   25360|
|SLC|   25323|
|JFK|   23484|
|DTW|   23310|
|SEA|   23074|
|MSP|   22385|
|MIA|   21805|
+---+--------+
only showing top 20 rows



### Find the Outgoing connections from the airport sorted in Desc. order.

In [23]:
airportGraph.outDegrees.orderBy(col('outDegree').desc()).show(20)

+---+---------+
| id|outDegree|
+---+---------+
|ATL|    91484|
|DFW|    68482|
|ORD|    64228|
|LAX|    54086|
|DEN|    53148|
|IAH|    43361|
|PHX|    40155|
|SFO|    39483|
|LAS|    33107|
|CLT|    28402|
|MCO|    28313|
|EWR|    27656|
|SLC|    25868|
|LGA|    25458|
|BOS|    25348|
|MSP|    24031|
|JFK|    23572|
|DTW|    23421|
|SEA|    23078|
|MIA|    21817|
+---+---------+
only showing top 20 rows



### Use motif finding to answer this question: which delays could we blame on SFO?
#### Hint: this practically means that SFO is a transit station

In [24]:
motifs = airportGraph.find("(a)-[ab]->(b); (b)-[bc]->(c)").filter("(b.id = 'SFO') and (ab.delay > 500 or bc.delay > 500) and bc.tripid > ab.tripid and bc.tripid < ab.tripid + 10000")
motifs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  bc|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Albuquerque, NM,...|[1020600, 0, 779,...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Albuquerque, NM,...|[1210815, -12, 77...|[San Francisco, C...|[1211508, 593, 22...|[New York, NY, US...|
|[Eureka, CA, USA,...|[1011635, -15, 21...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Eureka, CA, USA,...|[1012016, -4, 217...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Eureka, CA, USA,...|[1020531, -2, 217...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Eureka, CA, USA,...|[1020948, -11, 21...|[San Francisco, C...|[1021507, 536, 22...|[New York, NY, US...|
|[Eureka, CA, USA,...|[1021506, -3, 2


### Determine Airport Ranking in Desc. order using PageRank algorithm

In [25]:
rank = airportGraph.pageRank(resetProbability=0.15 , tol = 0.01)

In [26]:
rank.vertices.orderBy(desc("pagerank")).show()

+--------------+-----+-------+---+------------------+
|          City|State|Country| id|          pagerank|
+--------------+-----+-------+---+------------------+
|       Atlanta|   GA|    USA|ATL| 27.56698537736015|
|        Dallas|   TX|    USA|DFW|19.920443138775934|
|       Chicago|   IL|    USA|ORD|19.520004862975792|
|        Denver|   CO|    USA|DEN|14.480081708686479|
|   Los Angeles|   CA|    USA|LAX|13.348042929307379|
|       Houston|   TX|    USA|IAH|11.684904182067894|
| San Francisco|   CA|    USA|SFO|10.536220246352993|
|       Phoenix|   AZ|    USA|PHX|  9.94331232433538|
|Salt Lake City|   UT|    USA|SLC| 8.790784923145996|
|     Las Vegas|   NV|    USA|LAS|  8.08821931674483|
|       Seattle|   WA|    USA|SEA| 7.075432687383829|
|       Orlando|   FL|    USA|MCO| 6.814624297060419|
|     Charlotte|   NC|    USA|CLT| 6.810267338187121|
|        Newark|   NJ|    USA|EWR| 6.742979168118522|
|       Detroit|   MI|    USA|DTW| 6.452242478559098|
|      New York|   NY|    US

## Determine the most popular flights (single city hops)

In [27]:
popularTrips = airportGraph.edges.groupBy("src", "dst").agg(count("delay").alias("trips"))
popularTrips.show()

+---+---+-----+
|src|dst|trips|
+---+---+-----+
|ATL|GSP|  507|
|BQN|MCO|   90|
|CLE|SJU|   21|
|DSM|EWR|   71|
|EWR|STT|   98|
|FSD|ATL|   89|
|LAS|LIT|   90|
|LBB|DEN|  175|
|MCI|IAH|  576|
|MCI|MKE|  155|
|MDW|MEM|  173|
|ORD|PDX|  544|
|PBI|DCA|  148|
|PHL|MCO| 1208|
|ROC|CLE|   24|
|SJC|LIH|   52|
|SMF|BUR|  571|
|SNA|PHX| 1023|
|HRL|BRO|    1|
|AUS|ELP|  195|
+---+---+-----+
only showing top 20 rows



### Find and Save a Subragph that obtained from the following pattern:
#### The flight starts from an airport and return back to the same airport through 2 other airports.

## I spent so much time to run it so  I exectuted the runtime

In [ ]:
airportGraph.find("""(v1)-[ti]->(v2); 
                (v2)-[t2]->(v3);
                (v3)-[t3] ->(v1)""").show()